In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SFNet(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.user_pathway = config["user_pathway"][:]
        self.item_pathway = config["item_pathway"][:]
        self.combined_pathway = config["combined_pathway"][:]
        self.embedding_dim = config["embedding_dim"]

        self.user_embedding = nn.Embedding(
            num_embeddings=config["num_user_emb"],
            embedding_dim=self.embedding_dim,
            max_norm=1.0,
        )
        self.cup_size_embedding = nn.Embedding(
            num_embeddings=config["num_cup_size_emb"],
            embedding_dim=self.embedding_dim,
            max_norm=1.0,
        )
        self.item_embedding = nn.Embedding(
            num_embeddings=config["num_item_emb"],
            embedding_dim=self.embedding_dim,
            max_norm=1.0,
        )
        self.category_embedding = nn.Embedding(
            num_embeddings=config["num_category_emb"],
            embedding_dim=self.embedding_dim,
            max_norm=1.0,
        )

        # Customer pathway transformation
        # user_embedding_dim + cup_size_embedding_dim + num_user_numeric_features
        user_features_input_size = 2 * self.embedding_dim + config["num_user_numeric"]
        self.user_pathway.insert(0, user_features_input_size)
        self.user_transform_blocks = []
        for i in range(1, len(self.user_pathway)):
            self.user_transform_blocks.append(
                SkipBlock(
                    self.user_pathway[i - 1],
                    self.user_pathway[i],
                    config["activation"],
                )
            )
            self.user_transform_blocks.append(nn.Dropout(p=config["dropout"]))
        self.user_transform_blocks = nn.Sequential(*self.user_transform_blocks)

        # Article pathway transformation
        # item_embedding_dim + category_embedding_dim + num_item_numeric_features
        item_features_input_size = 2 * self.embedding_dim + config["num_item_numeric"]
        self.item_pathway.insert(0, item_features_input_size)
        self.item_transform_blocks = []
        for i in range(1, len(self.item_pathway)):
            self.item_transform_blocks.append(
                SkipBlock(
                    self.item_pathway[i - 1],
                    self.item_pathway[i],
                    config["activation"],
                )
            )
            self.item_transform_blocks.append(nn.Dropout(p=config["dropout"]))
        self.item_transform_blocks = nn.Sequential(*self.item_transform_blocks)

        # Combined top layer pathway
        # u = output dim of user_transform_blocks
        # t = output dim of item_transform_blocks
        # Pathway combination through [u, t, |u-t|, u*t]
        # Hence, input dimension will be 4*dim(u)
        combined_layer_input_size = 4 * self.user_pathway[-1]
        self.combined_pathway.insert(0, combined_layer_input_size)
        self.combined_blocks = []
        for i in range(1, len(self.combined_pathway)):
            self.combined_blocks.append(
                SkipBlock(
                    self.combined_pathway[i - 1],
                    self.combined_pathway[i],
                    config["activation"],
                )
            )
            self.combined_blocks.append(nn.Dropout(p=config["dropout"]))
        self.combined_blocks = nn.Sequential(*self.combined_blocks)

        # Linear transformation from last hidden layer to output
        self.hidden2output = nn.Linear(self.combined_pathway[-1], config["num_targets"])

    def forward(self, batch_input):

        # Customer Pathway
        user_emb = self.user_embedding(batch_input["user_id"])
        cup_size_emb = self.cup_size_embedding(batch_input["cup_size"])
        user_representation = torch.cat(
            [user_emb, cup_size_emb, batch_input["user_numeric"]], dim=-1
        )
        user_representation = self.user_transform_blocks(user_representation)

        # Article Pathway
        item_emb = self.item_embedding(batch_input["item_id"])
        category_emb = self.category_embedding(batch_input["category"])
        item_representation = torch.cat(
            [item_emb, category_emb, batch_input["item_numeric"]], dim=-1
        )
        item_representation = self.item_transform_blocks(item_representation)

        # Combine the pathways
        combined_representation = self.merge_representations(
            user_representation, item_representation
        )
        combined_representation = self.combined_blocks(combined_representation)

        # Output layer of logits
        logits = self.hidden2output(combined_representation)
        pred_probs = F.softmax(logits, dim=-1)

        return logits, pred_probs

    def merge_representations(self, u, v):
        """
        Combining two different representations via:
        - concatenation of the two representations
        - element-wise product u ∗ v
        - absolute element-wise difference |u-v|
        Link: https://arxiv.org/pdf/1705.02364.pdf
        """
        return torch.cat([u, v, torch.abs(u - v), u * v], dim=-1)


class SkipBlock(nn.Module):
    def __init__(self, input_dim, output_dim, activation):
        """
        Skip Connection for feed-forward block based on ResNet idea:
        Refer: 
        - Youtube: https://www.youtube.com/watch?v=ZILIbUvp5lk
        - Medium: https://medium.com/@14prakash/understanding-and-implementing-architectures-of-resnet-and-resnext-for-state-of-the-art-image-cf51669e1624 
                 Residual block function when the input and output dimensions are not same.

        """
        super().__init__()
        assert activation in [
            "relu",
            "tanh",
        ], "Please specify a valid activation funciton: relu or tanh"
        if activation == "relu":
            self.activation = F.relu
        elif activation == "tanh":
            self.activation = F.tanh

        self.inp_transform = nn.Linear(input_dim, output_dim)
        self.out_transform = nn.Linear(output_dim, output_dim)
        self.inp_projection = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        """
        y = x --> T1(x) --> ReLU(T1(x))
        z = ReLU(T2(y) + Projection(x))
        """
        y = self.activation(self.inp_transform(x))
        z = self.activation(self.out_transform(y) + self.inp_projection(x))
        return z


In [2]:
import json
import os
from typing import Dict

import _jsonnet
import torch
from sklearn import metrics
from torch.autograd import Variable


def load_config_from_json(config_file: str) -> Dict:
    # load configuration
    if not os.path.isfile(config_file):
        raise ValueError("given configuration file doesn't exist")
    with open(config_file, "r") as fio:
        config = fio.read()
        config = json.loads(_jsonnet.evaluate_snippet("", config))
    return config


def to_var(x, volatile=False):
    # To convert tensors to CUDA tensors if GPU is available
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)


def compute_metrics(target, pred_probs):
    """
    Computes metrics to report
    """
    pred_labels = pred_probs.argmax(-1)
    precision = metrics.precision_score(target, pred_labels, average="macro")
    recall = metrics.recall_score(target, pred_labels, average="macro")
    f1_score = metrics.f1_score(target, pred_labels, average="macro")
    accuracy = metrics.accuracy_score(target, pred_labels)
    auc = metrics.roc_auc_score(target, pred_probs, average="macro", multi_class="ovr")

    return precision, recall, f1_score, accuracy, auc


In [6]:
saved_model_path = "../runs/trial_2020-Jul-04-10-16-59"
model_config = load_config_from_json(
    os.path.join(saved_model_path, "config.jsonl")
)
model = SFNet(model_config["sfnet"])

In [16]:
import pandas as pd
df = pd.read_json("../data/modcloth_final_data_processed_train.jsonl", orient='records', lines=True)

array([ 0.2813063 ,  0.02400028,  0.15010627,  0.11314449,  0.18134221,
       -0.6971236 ,  0.43736464,  0.2947118 , -0.24761261, -0.16320565],
      dtype=float32)

In [19]:
item_ids = df["item_id"].unique()

In [22]:
item_embeddings = model.item_embedding(torch.tensor(item_ids))

In [30]:
from tensorboardX import SummaryWriter

writer = SummaryWriter()
writer.add_embedding(item_embeddings, metadata=item_ids.tolist())